In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf
from time import time
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

In [2]:
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()

In [3]:
# Print training set shapes
print("Training data: ",X_train.shape)
print("Labels of training data: ",Y_train.shape)
print("Unique labels: ", np.unique(Y_train))

Training data:  (60000, 28, 28)
Labels of training data:  (60000,)
Unique labels:  [0 1 2 3 4 5 6 7 8 9]


In [4]:
# Print test set shapes
print("Training data: ",X_test.shape)
print("Labels of training data: ",Y_test.shape)
print("Unique labels: ", np.unique(Y_test))

Training data:  (10000, 28, 28)
Labels of training data:  (10000,)
Unique labels:  [0 1 2 3 4 5 6 7 8 9]


In [5]:
#Scaling values
X_train, X_test = X_train/255.0, X_test/255.0

In [6]:
#convert labels to one hot encoding
Y_train = tf.keras.utils.to_categorical(Y_train)

# Count labels used in training set; categorise test set on same basis
# even if test set only uses subset of categories learning in training
K = len(Y_train[0])

Y_test = tf.keras.utils.to_categorical(Y_test, K)

In [7]:
#normalize each column : x_normed = x/ x.max(axis = 0)
max_train_data = X_train.max(axis=0)
X_train = X_train / max_train_data
X_test = X_test/max_train_data #use the train data maximum values

In [8]:
#Re-shape the image data in order to have a Conv2D layer
X_train = X_train.reshape(60000, 28,28,1)
X_test = X_test.reshape(10000, 28,28,1)

In [9]:
def create_model():
    model  = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=(5,5), input_shape=(28,28,1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, kernel_size=(5,5)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation = "relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(K, activation = tf.nn.softmax)
    ])

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics  = ['accuracy']
    )
    return model

In [10]:
model = KerasClassifier(model=create_model)

In [11]:
# define the grid search parameters
batch_size = [1000] #10, 100, 200, 300, 400, 500,
epochs = [5,10, 20]
param_grid = dict(batch_size=batch_size, epochs=epochs) #
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    #n_jobs=-1,
                    verbose=3,
                    return_train_score=True,
                    cv=3)
grid_result = grid.fit(X_train, Y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Epoch 1/5
40/40 [==============================] - 42s 1s/step - loss: 0.8718 - accuracy: 0.6874
Epoch 2/5
40/40 [==============================] - 41s 1s/step - loss: 0.5091 - accuracy: 0.8161
Epoch 3/5
40/40 [==============================] - 42s 1s/step - loss: 0.4330 - accuracy: 0.8464
Epoch 4/5
40/40 [==============================] - 41s 1s/step - loss: 0.3781 - accuracy: 0.8636
Epoch 5/5
40/40 [==============================] - 14s 337ms/step
[CV 1/3] END batch_size=1000, epochs=5;, score=(train=0.890, test=0.877) total time= 3.6min
Epoch 1/5
40/40 [==============================] - 43s 1s/step - loss: 0.8868 - accuracy: 0.6840
Epoch 2/5
40/40 [==============================] - 41s 1s/step - loss: 0.5240 - accuracy: 0.8080
Epoch 3/5
40/40 [==============================] - 41s 1s/step - loss: 0.4376 - accuracy: 0.8435
Epoch 4/5
40/40 [==============================] - 42s 1s/step - loss: 0.3876 - accuracy: 0.8607
Epoch 5

In [12]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.910383 using {'batch_size': 1000, 'epochs': 20}
0.877117 (0.000878) with: {'batch_size': 1000, 'epochs': 5}
0.895650 (0.004227) with: {'batch_size': 1000, 'epochs': 10}
0.910383 (0.001051) with: {'batch_size': 1000, 'epochs': 20}


In [14]:
def create_model():
    model  = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=(5,5), input_shape=(28,28,1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, kernel_size=(5,5)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, kernel_size=(5,5)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation = "relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(K, activation = tf.nn.softmax)
    ])

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics  = ['accuracy']
    )
    return model

In [15]:
# define the grid search parameters
batch_size = [1000] #10, 100, 200, 300, 400, 500,
epochs = [20]
param_grid = dict(batch_size=batch_size, epochs=epochs) #
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    #n_jobs=-1,
                    verbose=3,
                    return_train_score=True,
                    cv=3)
grid_result = grid.fit(X_train, Y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Epoch 1/20
40/40 [==============================] - 39s 958ms/step - loss: 0.8762 - accuracy: 0.6897
Epoch 2/20
40/40 [==============================] - 41s 1s/step - loss: 0.5121 - accuracy: 0.8153
Epoch 3/20
40/40 [==============================] - 41s 1s/step - loss: 0.4187 - accuracy: 0.8521
Epoch 4/20
40/40 [==============================] - 47s 1s/step - loss: 0.3774 - accuracy: 0.8636
Epoch 5/20
40/40 [==============================] - 42s 1s/step - loss: 0.3411 - accuracy: 0.8774
Epoch 6/20
40/40 [==============================] - 41s 1s/step - loss: 0.3123 - accuracy: 0.8867
Epoch 7/20
40/40 [==============================] - 37s 934ms/step - loss: 0.2963 - accuracy: 0.8923
Epoch 8/20
40/40 [==============================] - 42s 1s/step - loss: 0.2731 - accuracy: 0.9021
Epoch 9/20
40/40 [==============================] - 38s 958ms/step - loss: 0.2627 - accuracy: 0.9043
Epoch 10/20
40/40 [==============================]

In [16]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.909517 using {'batch_size': 1000, 'epochs': 20}
0.909517 (0.001862) with: {'batch_size': 1000, 'epochs': 20}
